In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np

In [2]:
df = pd.read_csv('/home/shane/Documents/thesis/output/final/failures.csv')
df

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,source,service,timestamp_dir,timestamp_site,success_count,success_pct,partial_count,partial_pct,major_count,major_pct,maintenance_count,maintenance_pct
0,slack,slack,2020-03-26 12:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
1,slack,slack,2020-03-26 13:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
2,slack,slack,2020-03-26 14:00:02,NaN,10,1.0,0,0.0,0,0.0,0,0.0
3,slack,slack,2020-03-26 15:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
4,slack,slack,2020-03-26 16:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
403998,cloud-google-cloud-platform,google-cloud,2020-06-10 19:00:02,NaN,1,1.0,0,0.0,0,0.0,0,0.0
403999,cloud-google-cloud-platform,google-cloud,2020-06-10 20:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
404000,cloud-google-cloud-platform,google-cloud,2020-06-10 21:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
404001,cloud-google-cloud-platform,google-cloud,2020-06-10 22:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0


### Mean of statuses per source/service

In [3]:
df_mean = df.groupby(['source','service']).agg(mean_success=('success_pct',np.mean), mean_partial=('partial_pct',np.mean), mean_major=('major_pct',np.mean), mean_maintenance=('maintenance_pct',np.mean))
df_mean

mean_success  mean_partial  mean_major  \
source        service                                              
atlassian     access          1.000000      0.000000    0.000000   
              bitbucket       0.994805      0.001622    0.000817   
              confluence      0.994570      0.004568    0.000318   
              developers      0.998779      0.000660    0.000429   
              jira-align      1.000000      0.000000    0.000000   
...                                ...           ...         ...   
downrightnow  typepad         1.000000      0.000000    0.000000   
              yahoomail       0.995968      0.004032    0.000000   
              youtube         1.000000      0.000000    0.000000   
github-status github          0.996964      0.002711    0.000325   
slack         slack           0.980163      0.019837    0.000000   

                          mean_maintenance  
source        service                       
atlassian     access              0.000000  
              bitbucket           0.002756  
              confluence          0.000545  
              developers          0.000132  
              jira-align          0.000000  
...                                    ...  
downrightnow  typepad             0.000000  
              yahoomail           0.000000  
              youtube             0.000000  
github-status github              0.000000  
slack         slack               0.000000  

[188 rows x 4 columns]

### Frequency of status changes per source/service

In [9]:
df_grouped = df.groupby(['source','service'])
df_grouped.head()

,source,service,timestamp_dir,timestamp_site,success_count,success_pct,partial_count,partial_pct,major_count,major_pct,maintenance_count,maintenance_pct
0,slack,slack,2020-03-26 12:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
1,slack,slack,2020-03-26 13:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
2,slack,slack,2020-03-26 14:00:02,NaN,10,1.0,0,0.0,0,0.0,0,0.0
3,slack,slack,2020-03-26 15:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
4,slack,slack,2020-03-26 16:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
381192,cloud-google-cloud-platform,google-cloud,2017-11-01 04:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
381193,cloud-google-cloud-platform,google-cloud,2017-11-01 05:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
381194,cloud-google-cloud-platform,google-cloud,2017-11-01 06:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
381195,cloud-google-cloud-platform,google-cloud,2017-11-01 07:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0


In [24]:
# first_values = df_grouped['success_pct'].first()
# first_values = first_values.reset_index()
# first_values = np.array(first_values['success_pct'])
# first_values

In [30]:
for name, group in df_grouped:
    if name[0] == 'cloudflare-status':
        continue
    x = group[group['success_pct'].diff() != 0].index.tolist()
    print(name, x)

('atlassian', 'access') [337456]
('atlassian', 'bitbucket') [337450, 337490, 337501, 338834, 338857, 339273, 339280, 340756, 340768, 341931, 341966, 344175, 344200, 345555, 345562, 347619, 347656, 349673, 349700, 350505, 350593, 351730, 351736, 352019, 352104, 352319, 352339, 352544, 352553, 352566, 352854, 352903, 355058, 355077, 357789, 357801, 359170, 359247, 359557, 359652, 360992, 361068]
('atlassian', 'confluence') [337447, 338526, 338545, 339188, 339266, 341258, 341319, 350504, 350596, 358465, 358554, 358563, 358569, 359087, 359183]
('atlassian', 'developers') [337452, 338128, 338184, 339052, 339076, 356169, 356179, 358284, 358316, 358716, 358734, 360551, 360760]
('atlassian', 'jira-align') [337449]
('atlassian', 'jira-core') [337448, 341617, 341707, 350312, 350324, 350506, 350591, 358550, 358571]
('atlassian', 'jira-service-desk') [337457, 341609, 341698, 350508, 350599, 358551, 358580, 359092, 359257]
('atlassian', 'jira-software') [337453, 341607, 341708, 347653, 347696, 3505

### MTBF per source/service